In [1]:
from p2pfl.node import Node
from p2pfl.learning.pytorch.mnist_examples.mnistfederated_dm import MnistFederatedDM
from p2pfl.learning.pytorch.mnist_examples.models.mlp import MLP
from collections import OrderedDict
import torch
import time


def test_convergence(x):
    n,r = x

    nodes = []
    
    for i in range(n):
        node = Node(MLP(),MnistFederatedDM(), simulation=False)
        node.start()
        nodes.append(node)

    # Node Connection
    for i in range(len(nodes)-1):
        nodes[i+1].connect_to(nodes[i].host,nodes[i].port)
        time.sleep(.1)

    time.sleep(3)     
        
    # Check if they are connected
    for node in nodes:
        assert len(node.neightboors) == n-1

    # Start Learning
    nodes[0].set_start_learning(rounds=r,epochs=1)

    # Wait 4 results
    while True:
        time.sleep(1)
        finish = True
        for f in [node.round is None for node in nodes]:
            finish = finish and f

        if finish:
            break


    # Validamos Modelos obtenidos sean iguales
    model = None
    first = True
    for node in nodes:
        if first:
            model = node.learner.get_parameters()
            first = False
        else:
            for layer in model:
                a = torch.round(model[layer], decimals=2)
                b = torch.round(node.learner.get_parameters()[layer], decimals=2)
                assert torch.eq(a, b).all()

    # Cerrar
    for node in nodes:
        node.stop()
        time.sleep(.1) #Esperar por la asincronía
    
    return nodes
        
nodes = test_convergence((4,2))

Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:35963
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:54353
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:49183
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:54253
INFO:root:('127.0.0.1', 54353) Connected to 127.0.0.1:35963
INFO:root:('127.0.0.1', 35963) Conexión aceptada con 127.0.0.1:54353
DEBUG:root:('127.0.0.1', 35963) (NodeConnection Send) Exception: can't concat str to bytes
DEBUG:root:('127.0.0.1', 54353) (NodeConnection Send) Exception: can't concat str to bytes
ERROR:root:can't concat str to bytes
Traceback (most recent call last):
  File "/home/pedro/Documents/federated_learning_p2p/p2pfl/node_connection.py", line 276, in send
    data = self.aes_cipher.encrypt( self.aes_cipher.encode_with_padding(data))
  File "/home/pedro/Documents/federated_learning_p2p/p2pfl/encrypter.py", line 108, in encode_with_padding
    msg_pad =  msg + (sel

AssertionError: 

In [ ]:
from p2pfl.learning.pytorch.mnist_examples.mnistfederated_dm import MnistFederatedDM
from p2pfl.learning.pytorch.mnist_examples.models.cnn import CNN
from p2pfl.learning.pytorch.mnist_examples.models.mlp import MLP
from p2pfl.node import Node
import pytest
import time
import threading

nodes = []
        
def test_node_down_on_learning(n):

    # Node Creation
    nodes = []
    for i in range(n):
        node = Node(MLP(),MnistFederatedDM())
        node.start()
        nodes.append(node)

    # Node Connection
    for i in range(len(nodes)-1):
        nodes[i+1].connect_to(nodes[i].host,nodes[i].port)
        time.sleep(1)

    # Check if they are connected
    for node in nodes:
        print(len(node.neightboors))
        assert len(node.neightboors) == n-1

    # Start Learning
    nodes[0].set_start_learning(rounds=2,epochs=0)

    # Stopping node
    time.sleep(0.3)
    nodes[1].stop()
    # Wait 4 results
    while True:
        time.sleep(1)
        finish = True
        for f in [node.round is None for node in nodes]:
            finish = finish and f

        if finish:
            break

    for node in nodes:
        node.stop()


nodes = []
for _ in range(6):
    test_node_down_on_learning(4)
    break


Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:53999
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:58131
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:44173
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:46897
INFO:root:('127.0.0.1', 58131) Connected to 127.0.0.1:53999
INFO:root:('127.0.0.1', 53999) Conexión aceptada con 127.0.0.1:58131
INFO:root:('127.0.0.1', 44173) Connected to 127.0.0.1:58131
INFO:root:('127.0.0.1', 58131) Conexión aceptada con 127.0.0.1:44173
INFO:root:('127.0.0.1', 53999) Connected to 127.0.0.1:44173
INFO:root:('127.0.0.1', 44173) Conexión aceptada con 127.0.0.1:53999
INFO:root:('127.0.0.1', 46897) Connected to 127.0.0.1:44173
INFO:root:('127.0.0.1', 44173) Conexión aceptada con 127.0.0.1:46897
INFO:root:('127.0.0.1', 58131) Connected to 127.0.0.1:46897
INFO:root:('127.0.0.1', 46897) Conexión aceptada con 127.0.0.1:58131
INFO:root:('127.0.0.1', 53999) Connected to 127

In [72]:
import base64
import hashlib
from Crypto import Random
from Crypto.Cipher import AES

class AESCipher(object):
    
    def get_block_size():
        return AES.block_size
    
    def __init__(self, key): 
        self.bs = AES.block_size
        self.key = hashlib.sha256(key).digest()

    def encrypt(self, raw):
        cipher = AES.new(self.key, AES.MODE_ECB)
        return cipher.encrypt(raw)

    def decrypt(self, enc):
        cipher = AES.new(self.key, AES.MODE_ECB)
        return cipher.decrypt(enc)
    
    def encode_with_padding(self, msg):
        # Siempre añade apdding (si el tamaño de bloque coincide agrega 1 mas para evitar conflictos)
        msg_pad =  msg + (self.bs - len(msg) % self.bs) * chr(self.bs - len(msg) % self.bs)
        return msg_pad.encode("utf-8")

    def decode_without_padding(self,msg):
        msg_without_padding = msg[:-ord(msg[len(msg) - 1:])]
        return msg_without_padding.decode("utf-8")


In [73]:
a=AESCipher(key = b'Sixteen byte key')
b=AESCipher(key = b'Sixteen byte key')

In [50]:
from p2pfl.node import Node
from p2pfl.learning.pytorch.mnist_examples.mnistfederated_dm import MnistFederatedDM
from p2pfl.learning.pytorch.mnist_examples.models.mlp import MLP
from collections import OrderedDict
import torch
import time

n = Node(MLP(),MnistFederatedDM())
params = n.learner.encode_parameters()
assert params == a.decrypt(a.encrypt(params))



Train: 54000 Val:6000 Test:10000


In [71]:
string="iiiiiiiiiiiiiiii"

print(len(string.encode("utf-8")))

res = a.encrypt(a.encode_with_padding(string))

print(res)

res = a.decrypt(res)

print(res)

res = a.decode_without_padding(res)

print(res)

assert string == res

16
b'\x01Xf;\xc6H[U\x84}\xad\xbc4\xe6\xff\xcf\xd6/\x16\xf7\\c\x9f\xe1v\xe42^\x8fao\xa6'
b'iiiiiiiiiiiiiiii\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10'
iiiiiiiiiiiiiiii


In [47]:
res2 = a.decrypt(res)

TypeError: Object type <class 'str'> cannot be passed to C code

In [26]:
print(len(res))
print(len(res2))

544640
544640


In [29]:
print(params==res2)

True


In [28]:
print(res2)

b'\x80\x04\x95\x87\x00\x00\x00\x00\x00\x00\x00]\x94(\x8c\x15numpy.core.multiarray\x94\x8c\x0c_reconstruct\x94\x93\x94\x8c\x05numpy\x94\x8c\x07ndarray\x94\x93\x94K\x00\x85\x94C\x01b\x94\x87\x94R\x94(K\x01K\x80M\x10\x03\x86\x94h\x04\x8c\x05dtype\x94\x93\x94\x8c\x02f4\x94\x89\x88\x87\x94R\x94(K\x03\x8c\x01<\x94NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00t\x94b\x89B\x00 \x06\x00\\ \\\xbc \x81\x86\xbc\xe3)\xe3\xbcN\\\xe9\xbc\xbc\xc1\xb6\xbc\xb0\xff\x0c\xbd\xfc\x00E\xbc\x98\x0e\xf3\xbcnj\xab;r\xf9\x0f\xbd\x9e\x1a\x91\xbc\x01Yr;\xber\xdd<%\xbb\x03;J\xee\x92\xb9j.\x0c=\x81@\xbf;%\x9bj\xba(\x8d\xfe\xbc%\xa8\x92<sv\x8a\xbcJ\xf7\x19\xbc%\xcd\x819\x9e@\xef<e\x05\x0c\xbc\x1c\x94\x94<\xc1\x01\xc4\xbb\x16\xad\x0f\xbd\x88\xb8\xfd\xbc\xe7\x07\xb6<\x8a}\x88\xbc\xab\xb1\x00\xbd\x01\xdb`\xbc\xa5\x8fw\xbc\n\x1e\xe5<\x93\xe6\x02<J^\xe0\xbb%\xabF\xbb8\xe1\xe2\xbb\x83Q\n\xbdg7\x03=J`H\xba\x93\xed\xb2\xbc\x84\xde\x08\xbdE\xf7\x03\xbd\x01\xcb6;wR\x8e;n\xe1\xa4:~E\x92<\xb7\x17\x03\xba\x13\x19J\xbb\x00L\x97\xbc\xca

In [3]:
import Crypto.Cipher.AES


In [4]:
ord("1")

49

In [6]:
"a"+chr(1)

'a\x01'

In [82]:
BUFFER_SIZE = 1 # Try to avoid model sobresegmentation
rest = BUFFER_SIZE % AESCipher.get_block_size()
if rest != 0:
    new_value = BUFFER_SIZE + AESCipher.get_block_size() - rest
    print("Changing buffer size to %d. %d is incompatible with the AES block size", BUFFER_SIZE, new_value)
    BUFFER_SIZE = new_value

Changing buffer size to %d. %d is incompatible with the AES block size 1 16


In [4]:
from Crypto import Random
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
import base64
import hashlib
from Crypto import Random
from Crypto.Cipher import AES


priv= RSA.generate(1024)
pub=priv.publickey()

cipher = PKCS1_OAEP.new(priv)

encripted = cipher.encrypt(b"0")

cipher = PKCS1_OAEP.new(pub)

cipher.decrypt(encripted)






TypeError: This is not a private key

In [8]:
from p2pfl.communication_protocol import CommunicationProtocol
from p2pfl.encrypter import RSACipher
from p2pfl.encrypter import AESCipher
from p2pfl.learning.pytorch.lightninglearner import LightningLearner

from p2pfl.learning.pytorch.mnist_examples.models.mlp import MLP



#############################
#    RSA Encryption Test    #
#############################

def test_aes_encription_decription1():
    cipher = AESCipher()
    message="zzZZZZ!"
    encoded_mesage = cipher.padding_to_encoded_text(message.encode("utf-8"))
    encrypted_message = cipher.encrypt(encoded_mesage)
    decrypted_message = cipher.decrypt(encrypted_message)
    decoded_message = " ".join(decrypted_message.decode("utf-8").split(" "))
    
    assert message == decoded_message


test_aes_encription_decription_model()

[b'PARAMS\x80\x04\x95\x87\x00\x00\x00\x00\x00\x00\x00]\x94(\x8c\x15numpy.core.multiarray\x94\x8c\x0c_reconstruct\x94\x93\x94\x8c\x05numpy\x94\x8c\x07ndarray\x94\x93\x94K\x00\x85\x94C\x01b\x94\x87\x94R\x94(K\x01K\x80M\x10\x03\x86\x94h\x04\x8c\x05dtype\x94\x93\x94\x8c\x02f4\x94\x89\x88\x87\x94R\x94(K\x03\x8c\x01<\x94NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00t\x94b\x89B\x00 \x06\x00\\ \\\xbc \x81\x86\xbc\xe3)\xe3\xbcN\\\xe9\xbc\xbc\xc1\xb6\xbc\xb0\xff\x0c\xbd\xfc\x00E\xbc\x98\x0e\xf3\xbcnj\xab;r\xf9\x0f\xbd\x9e\x1a\x91\xbc\x01Yr;\xber\xdd<%\xbb\x03;J\xee\x92\xb9j.\x0c=\x81@\xbf;%\x9bj\xba(\x8d\xfe\xbc%\xa8\x92<sv\x8a\xbcJ\xf7\x19\xbc%\xcd\x819\x9e@\xef<e\x05\x0c\xbc\x1c\x94\x94<\xc1\x01\xc4\xbb\x16\xad\x0f\xbd\x88\xb8\xfd\xbc\xe7\x07\xb6<\x8a}\x88\xbc\xab\xb1\x00\xbd\x01\xdb`\xbc\xa5\x8fw\xbc\n\x1e\xe5<\x93\xe6\x02<J^\xe0\xbb%\xabF\xbb8\xe1\xe2\xbb\x83Q\n\xbdg7\x03=J`H\xba\x93\xed\xb2\xbc\x84\xde\x08\xbdE\xf7\x03\xbd\x01\xcb6;wR\x8e;n\xe1\xa4:~E\x92<\xb7\x17\x03\xba\x13\x19J\xbb\x00L\x97\

In [2]:
from Crypto.Random import get_random_bytes
len(get_random_bytes(16))

16

In [10]:
from Crypto.PublicKey import RSA
len((RSA.generate(1024)).publickey().export_key())



271

In [6]:
"   a    a   ".split(" ")

['', '', '', 'a', '', '', '', 'a', '', '', '']

In [5]:
len("  ".encode("utf-8"))

2